In [3]:
import json
import pandas as pd
import numpy as np
import re

In [2]:
# pip install XlsxWriter

Note: you may need to restart the kernel to use updated packages.


In [22]:
import re
import pandas as pd

# starting with raw_ofac_list
df = pd.read_excel(r'C:\Users\wasi\Desktop\wdawda.xlsx')

# Extract content of the first outer parentheses
def extract_outer_first_parentheses(text):
    stack = []
    start_index = None

    for i, char in enumerate(text):
        if char == '(':
            if start_index is None:
                start_index = i
            stack.append(i)
        elif char == ')':
            if stack:
                stack.pop()
                if not stack:
                    return text[start_index + 1:i]
    return None

# Extract name and initial aliases based on conditions
def extract_name_aliases(text):
    name = ''
    aliases = []

    first_paren = text.find('(')
    if first_paren == -1:
        return '', []

    first_content = extract_outer_first_parentheses(text[first_paren:])
    if not first_content:
        return '', []

    # Condition 1
    if ':' in first_content and 'Linked To:' not in first_content:
        name = text[:first_paren].strip()
        aliases.append(first_content.strip())
        
        remaining_text = text[first_paren + len(first_content) + 2:]
        next_paren_content = extract_outer_first_parentheses(remaining_text)
        if next_paren_content and ('a.k.a' in next_paren_content or 'f.k.a' in next_paren_content):
            for part in next_paren_content.split(';'):
                aliases.append(part.strip())
        return name, aliases

    # Condition 2
    elif ':' not in first_content and 'Linked To:' not in first_content:
        if 'a.k.a' in first_content or 'f.k.a' in first_content:
            name = text[:first_paren].strip()
            for part in first_content.split(';'):
                aliases.append(part.strip())
            return name, aliases
        else:
            return '', []

    # Condition 3
    else:
        return '', []

# Clean aliases: remove quotes, extract nested parentheses, split by ';'
def clean_nested_parentheses_from_aliases(aliases):
    final_aliases = []

    for alias in aliases:
        # Find all parentheses content
        nested = re.findall(r'\((.*?)\)', alias)
        # Remove parentheses content from original alias
        cleaned_alias = re.sub(r'\(.*?\)', '', alias).strip()
        cleaned_alias = cleaned_alias.replace('"', '')
        # Split by ';' and add each part
        if cleaned_alias:
            for part in cleaned_alias.split(';'):
                part_clean = part.strip()
                if part_clean:
                    final_aliases.append(part_clean)
        # Add nested content as separate aliases (also remove quotes)
        for item in nested:
            item_clean = item.strip().replace('"', '')
            if item_clean:
                final_aliases.append(item_clean)
    
    return final_aliases



def extract_final_name_aliases_joined(text):
    name, aliases = extract_name_aliases(text)
    if aliases:
        aliases = clean_nested_parentheses_from_aliases(aliases)
        aliases = ';'.join(aliases)  # join by ;
    else:
        aliases = ''
    return name, aliases



# Apply the extraction function to each row
df[['name', 'alias']] = df['list'].apply(lambda x: pd.Series(extract_final_name_aliases_joined(x)))

# exporting to check file
df.to_excel('df_check.xlsx', index=False)

# Show results
df.head()


,list,source,name,alias
0,"ESQUEDA NIETO, Francisco Daniel (a.k.a. ESQUED...",https://ofac.treasury.gov/recent-actions/20250811,"ESQUEDA NIETO, Francisco Daniel","a.k.a. ESQUEDA, Franky;a.k.a. ESKEDA, Franki;a..."
1,"ALIZADEH HOSHYAR, Javad (Arabic: جواد عليزاده ...",https://ofac.treasury.gov/recent-actions/20250811,"ALIZADEH HOSHYAR, Javad",Arabic: جواد عليزاده هوشيار;a.k.a. ALIZADE HOU...
2,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD...,https://ofac.treasury.gov/recent-actions/20250811,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,a.k.a. CHANGZHOU JOMARS INDUSTRIAL AUTOMATION ...
3,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD...,https://ofac.treasury.gov/recent-actions/20250811,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,a.k.a. CHANGZHOU JOMARS INDUSTRIAL AUTOMATION ...
4,"DE MORAES, Alexandre, Brazil; DOB 03 Dec 1968;...",https://ofac.treasury.gov/recent-actions/20250811,,


In [6]:
import pandas as pd

url = "https://data.trade.gov/downloadable_consolidated_screening_list/v1/consolidated.csv"

df_csv = pd.read_csv(url)

# Display first 5 rows
df_csv.head()

,_id,source,entity_number,type,programs,name,title,addresses,federal_register_notice,start_date,...,vessel_owner,remarks,source_list_url,alt_names,citizenships,dates_of_birth,nationalities,places_of_birth,source_information_url,ids
0,b6e959b59898e975daaae8fae3a8da1ec522f9f888cafc...,Entity List (EL) - Bureau of Industry and Secu...,NaN,NaN,NaN,Abdul Satar Ghoura,NaN,"501, 5th Floor, Amanullah Sancharaki Market Op...",76 FR 71867,2011-11-21,...,NaN,NaN,https://www.bis.doc.gov/index.php/policy-guida...,NaN,NaN,NaN,NaN,NaN,https://www.bis.doc.gov/index.php/policy-guida...,NaN
1,15268,Capta List (CAP) - Treasury Department,15268.0,Entity,561-Related,BANK OF KUNLUN CO LTD,NaN,"172 Xibin Rd, Ranghulu District, (Daqing, Heil...",NaN,NaN,...,NaN,United States financial institutions are prohi...,https://sanctionslist.ofac.treas.gov/Home/Cons...,KARAMAY URBAN CREDIT COOPERATIVES; KARAMAY CIT...,NaN,NaN,NaN,NaN,https://ofac.treasury.gov/other-ofac-sanctions...,"SWIFT/BIC, CKLBCNBJ"
2,278be3e06d711cd8c6ea146bcd625d9f17cc901b980ea8...,Denied Persons List (DPL) - Bureau of Industry...,NaN,NaN,NaN,I. ASH,NaN,UPON THE DATE OF THE ORDER INCARCERATED AT USM...,68 F.R. 38290 6/27/03 71 F.R. 38843 7/10/06 72...,2003-06-19,...,NaN,FEDERAL REGISTER NOTICE UPDATED,https://www.bis.doc.gov/index.php/the-denied-p...,NaN,NaN,NaN,NaN,NaN,https://www.bis.doc.gov/index.php/policy-guida...,NaN
3,66f6ab3f26234b75a9093fce79ef0ce0,Military End User (MEU) List - Bureau of Indus...,NaN,NaN,NaN,Academy of Aerospace Solid Propulsion Technolo...,NaN,"Tian Wang te Zi #1, Baqiao District, Xian, Chi...",85 FR 83799,2020-12-23,...,NaN,NaN,https://www.bis.doc.gov/index.php/policy-guida...,NaN,NaN,NaN,NaN,NaN,https://www.bis.doc.gov/index.php/policy-guida...,NaN
4,26182,Foreign Sanctions Evaders (FSE) - Treasury Dep...,26182.0,Individual,FSE-IR,Evren KAYAKIRAN,NaN,TR,NaN,NaN,...,NaN,NaN,https://sanctionslist.ofac.treas.gov/Home/Cons...,NaN,TR,1980-02-08,NaN,NaN,https://ofac.treasury.gov/other-ofac-sanctions...,"Passport, U00242309, TR; Gender, Male"


In [7]:
df_csv.shape

(23318, 29)

In [7]:
# df_csv.to_csv('ofac.csv')

In [23]:
df_check = pd.read_excel('df_check.xlsx')
df_check

,list,source,name,alias
0,"ESQUEDA NIETO, Francisco Daniel (a.k.a. ESQUED...",https://ofac.treasury.gov/recent-actions/20250811,"ESQUEDA NIETO, Francisco Daniel","a.k.a. ESQUEDA, Franky;a.k.a. ESKEDA, Franki;a..."
1,"ALIZADEH HOSHYAR, Javad (Arabic: جواد عليزاده ...",https://ofac.treasury.gov/recent-actions/20250811,"ALIZADEH HOSHYAR, Javad",Arabic: جواد عليزاده هوشيار;a.k.a. ALIZADE HOU...
2,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD...,https://ofac.treasury.gov/recent-actions/20250811,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,a.k.a. CHANGZHOU JOMARS INDUSTRIAL AUTOMATION ...
3,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD...,https://ofac.treasury.gov/recent-actions/20250811,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,a.k.a. CHANGZHOU JOMARS INDUSTRIAL AUTOMATION ...
4,"DE MORAES, Alexandre, Brazil; DOB 03 Dec 1968;...",https://ofac.treasury.gov/recent-actions/20250811,NaN,NaN
5,"DE MORAES, Alexandre, Brazil; DOB 03 Dec 1968;...",https://ofac.treasury.gov/recent-actions/20250811,NaN,NaN


In [9]:
# checking if df_check names found in df_csv
import pandas as pd

def match_names(df_check, df_csv):
    name_matched_list = []
    id_list = []
    
    for name in df_check["name"]:
        matches = df_csv[df_csv["name"] == name]
        
        if len(matches) == 1:
            name_matched_list.append(matches.iloc[0]["name"])
            id_list.append(matches.iloc[0]["_id"])
        
        elif len(matches) > 1:
            name_matched_list.append("multiple_matches_need_to_check")
            id_list.append(None)
        
        else:
            name_matched_list.append(None)
            id_list.append(None)
    
    df_check["name_matched"] = name_matched_list
    df_check["id"] = id_list
    return df_check
# applying
df_check = match_names(df_check, df_csv)
df_check



,list,source,name,aliases,name_matched,id
0,"ESQUEDA NIETO, Francisco Daniel (a.k.a. ESQUED...",https://ofac.treasury.gov/recent-actions/20250811,"ESQUEDA NIETO, Francisco Daniel","a.k.a. ESQUEDA, Franky;a.k.a. ESKEDA, Franki;a...","ESQUEDA NIETO, Francisco Daniel",54505
1,"ALIZADEH HOSHYAR, Javad (Arabic: جواد عليزاده ...",https://ofac.treasury.gov/recent-actions/20250811,"ALIZADEH HOSHYAR, Javad",Arabic: جواد عليزاده هوشيار;a.k.a. ALIZADE HOU...,"ALIZADEH HOSHYAR, Javad",54930
2,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD...,https://ofac.treasury.gov/recent-actions/20250811,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,a.k.a. CHANGZHOU JOMARS INDUSTRIAL AUTOMATION ...,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,54945


In [10]:
# if all values not found recheck the df_check
df_check.to_excel('df_checkv2.xlsx', index=False)

In [24]:
df_checkv2 = pd.read_excel('df_checkv2.xlsx', dtype={'id': 'Int64'})
df_checkv2

,list,source,name,alias,name_matched,id
0,"ESQUEDA NIETO, Francisco Daniel (a.k.a. ESQUED...",https://ofac.treasury.gov/recent-actions/20250811,"ESQUEDA NIETO, Francisco Daniel","a.k.a. ESQUEDA, Franky;a.k.a. ESKEDA, Franki;a...","ESQUEDA NIETO, Francisco Daniel",54505
1,"ALIZADEH HOSHYAR, Javad (Arabic: جواد عليزاده ...",https://ofac.treasury.gov/recent-actions/20250811,"ALIZADEH HOSHYAR, Javad",Arabic: جواد عليزاده هوشيار;a.k.a. ALIZADE HOU...,"ALIZADEH HOSHYAR, Javad",54930
2,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD...,https://ofac.treasury.gov/recent-actions/20250811,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,a.k.a. CHANGZHOU JOMARS INDUSTRIAL AUTOMATION ...,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,54945


In [12]:
df_checkv2.shape

(3, 6)

In [25]:
# combining id and name column
df_checkv2["id-name"] = df_checkv2["id"].astype("string").fillna('') + "-" + df_checkv2["name"].fillna('').astype(str)
df_checkv2

,list,source,name,alias,name_matched,id,id-name
0,"ESQUEDA NIETO, Francisco Daniel (a.k.a. ESQUED...",https://ofac.treasury.gov/recent-actions/20250811,"ESQUEDA NIETO, Francisco Daniel","a.k.a. ESQUEDA, Franky;a.k.a. ESKEDA, Franki;a...","ESQUEDA NIETO, Francisco Daniel",54505,"54505-ESQUEDA NIETO, Francisco Daniel"
1,"ALIZADEH HOSHYAR, Javad (Arabic: جواد عليزاده ...",https://ofac.treasury.gov/recent-actions/20250811,"ALIZADEH HOSHYAR, Javad",Arabic: جواد عليزاده هوشيار;a.k.a. ALIZADE HOU...,"ALIZADEH HOSHYAR, Javad",54930,"54930-ALIZADEH HOSHYAR, Javad"
2,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD...,https://ofac.treasury.gov/recent-actions/20250811,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,a.k.a. CHANGZHOU JOMARS INDUSTRIAL AUTOMATION ...,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,54945,54945-CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION ...


In [14]:
# pip install pandas openpyxl

In [15]:
# OFAC_data_check = pd.read_excel(r'C:\Users\wasia\Desktop\OFAC data listing\ofac_data_check.xlsx')
# OFAC_data_check.Alias = OFAC_data_check.Alias.fillna("")
# OFAC_data_check

In [14]:

# Importing the json ofac data
import requests

url = "https://data.trade.gov/downloadable_consolidated_screening_list/v1/consolidated.json"  # clean URL without query tags
resp = requests.get(url)
resp.raise_for_status()

data = resp.json()

# Again, adjust 'entries' to match the JSON structure you're working with
df_json = pd.json_normalize(data,"results")

df_json.head()


,addresses,alt_names,entity_number,id,ids,name,programs,remarks,source,source_information_url,...,gross_tonnage,gross_registered_tonnage,vessel_flag,vessel_owner,title,citizenships,dates_of_birth,country,places_of_birth,nationalities
0,"[{'address': '172 Xibin Rd, Ranghulu District,...","[KARAMAY URBAN CREDIT COOPERATIVES, KARAMAY CI...",15268,15268,"[{'type': 'SWIFT/BIC', 'number': 'CKLBCNBJ'}]",BANK OF KUNLUN CO LTD,[561-Related],United States financial institutions are prohi...,Capta List (CAP) - Treasury Department,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'address': '31 Jinrong Street, Xicheng Distr...","[CHINA TELECOM, CHINA TELECOM CO., LTD, CHINA ...",30882,30882,"[{'type': 'Unified Social Credit Code (USCC)',...",China Telecom Corporation Limited,[CMIC-EO13959],None,Non-SDN Chinese Military-Industrial Complex Co...,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[{'address': '5 Landianchang South Road Haidia...,"[Aero Engine Corp of China, AECC, CHINA AVIATI...",30902,30902,"[{'type': 'Unified Social Credit Code (USCC)',...",Aero Engine Corporation of China,[CMIC-EO13959],None,Non-SDN Chinese Military-Industrial Complex Co...,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'address': '60th Floor, The Center, 99 Queen...",[CHINA MOBILE LTD],30923,30923,"[{'type': 'Equity Ticker', 'number': 'CHL US'}...",China Mobile Limited,[CMIC-EO13959],None,Non-SDN Chinese Military-Industrial Complex Co...,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[{'address': '75th Floor, The Center, 99 Queen...",[China Unicom Limited],30924,30924,"[{'type': 'Equity Ticker', 'number': 'CHU US'}...",China Unicom (Hong Kong) Limited,[CMIC-EO13959],None,Non-SDN Chinese Military-Industrial Complex Co...,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# with open(r'C:\UID Listing\OFAC.json', 'r', encoding='utf-8') as f:
#     data = json.load(f)
# df_all_data = pd.json_normalize(data['results'])
# df_all_data.head()



In [15]:
df_json.shape

(23318, 30)

In [16]:
# combining id and name column
df_json["ID-NAME"] = df_json["id"] + "-" + df_json["name"]
df_json.head()

,addresses,alt_names,entity_number,id,ids,name,programs,remarks,source,source_information_url,...,gross_registered_tonnage,vessel_flag,vessel_owner,title,citizenships,dates_of_birth,country,places_of_birth,nationalities,ID-NAME
0,"[{'address': '172 Xibin Rd, Ranghulu District,...","[KARAMAY URBAN CREDIT COOPERATIVES, KARAMAY CI...",15268,15268,"[{'type': 'SWIFT/BIC', 'number': 'CKLBCNBJ'}]",BANK OF KUNLUN CO LTD,[561-Related],United States financial institutions are prohi...,Capta List (CAP) - Treasury Department,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15268-BANK OF KUNLUN CO LTD
1,"[{'address': '31 Jinrong Street, Xicheng Distr...","[CHINA TELECOM, CHINA TELECOM CO., LTD, CHINA ...",30882,30882,"[{'type': 'Unified Social Credit Code (USCC)',...",China Telecom Corporation Limited,[CMIC-EO13959],None,Non-SDN Chinese Military-Industrial Complex Co...,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30882-China Telecom Corporation Limited
2,[{'address': '5 Landianchang South Road Haidia...,"[Aero Engine Corp of China, AECC, CHINA AVIATI...",30902,30902,"[{'type': 'Unified Social Credit Code (USCC)',...",Aero Engine Corporation of China,[CMIC-EO13959],None,Non-SDN Chinese Military-Industrial Complex Co...,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30902-Aero Engine Corporation of China
3,"[{'address': '60th Floor, The Center, 99 Queen...",[CHINA MOBILE LTD],30923,30923,"[{'type': 'Equity Ticker', 'number': 'CHL US'}...",China Mobile Limited,[CMIC-EO13959],None,Non-SDN Chinese Military-Industrial Complex Co...,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30923-China Mobile Limited
4,"[{'address': '75th Floor, The Center, 99 Queen...",[China Unicom Limited],30924,30924,"[{'type': 'Equity Ticker', 'number': 'CHU US'}...",China Unicom (Hong Kong) Limited,[CMIC-EO13959],None,Non-SDN Chinese Military-Industrial Complex Co...,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30924-China Unicom (Hong Kong) Limited


In [17]:
df_json.shape

(23318, 31)

In [21]:
# df_json.to_excel('df_json.xlsx')

In [84]:

# filtering checking df_check in df_json
filtered_df = df_json[df_json['ID-NAME'].isin(df_checkv2['id-name'])]



# Create a mapping dictionary from df_check
name_to_source = dict(zip(df_checkv2['id-name'], df_checkv2['source']))
idname_to_alias = dict(zip(df_checkv2['id-name'], df_checkv2['alias']))


# Add the 'Source' column to filtered_df2 using map
filtered_df['source'] = filtered_df['ID-NAME'].map(name_to_source)
filtered_df['alias2'] = filtered_df['ID-NAME'].map(idname_to_alias)
filtered_df

C:\Users\wasi\AppData\Local\Temp\ipykernel_11036\1946133589.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['source'] = filtered_df['ID-NAME'].map(name_to_source)
C:\Users\wasi\AppData\Local\Temp\ipykernel_11036\1946133589.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['alias2'] = filtered_df['ID-NAME'].map(idname_to_alias)


,addresses,alt_names,entity_number,id,ids,name,programs,remarks,source,source_information_url,...,vessel_flag,vessel_owner,title,citizenships,dates_of_birth,country,places_of_birth,nationalities,ID-NAME,alias2
22572,"[{'city': 'Nuevo Laredo', 'country': 'MX', 'ad...","[ESQUEDA, Franky, Franky De La Joya, ESKEDA, F...",54505,54505,"[{'type': 'C.U.R.P.', 'number': 'EUNF941023HTS...","ESQUEDA NIETO, Francisco Daniel","[ILLICIT-DRUGS-EO14059, SDGT]",(Linked To: CARTEL DEL NORESTE),https://ofac.treasury.gov/recent-actions/20250811,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[1994-10-23],NaN,"[Nuevo Laredo, Tamaulipas, Mexico]",[MX],"54505-ESQUEDA NIETO, Francisco Daniel","a.k.a. ESQUEDA, Franky;a.k.a. ESKEDA, Franki;a..."
22745,"[{'country': 'IR', 'address': None, 'city': No...","[ALIZADE HOUSHYAR, Javad, ALIZADEH-HOSHIAR, Ja...",54930,54930,"[{'type': 'National ID No.', 'number': '137656...","ALIZADEH HOSHYAR, Javad","[NPWMD, IFSR]",(Linked To: CONTROL AFZAR TABRIZ CO LTD),https://ofac.treasury.gov/recent-actions/20250811,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[1976-08-24],NaN,[Iran],[IR],"54930-ALIZADEH HOSHYAR, Javad",Arabic: جواد عليزاده هوشيار;a.k.a. ALIZADE HOU...
22755,"[{'city': 'Changzhou', 'country': 'CN', 'addre...",[CHANGZHOU JOMARS INDUSTRIAL AUTOMATION CO LTD...,54945,54945,"[{'type': 'Unified Social Credit Code (USCC)',...",CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,"[IFSR, NPWMD]",(Linked To: JOEMARS MACHINERY AND ELECTRIC IND...,https://ofac.treasury.gov/recent-actions/20250811,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[],NaN,[],[],54945-CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION ...,a.k.a. CHANGZHOU JOMARS INDUSTRIAL AUTOMATION ...


In [87]:
filtered_df

,addresses,alt_names,entity_number,id,ids,name,programs,remarks,source,source_information_url,...,vessel_flag,vessel_owner,title,citizenships,dates_of_birth,country,places_of_birth,nationalities,ID-NAME,alias2
22572,"[{'city': 'Nuevo Laredo', 'country': 'MX', 'ad...","[ESQUEDA, Franky, Franky De La Joya, ESKEDA, F...",54505,54505,"[{'type': 'C.U.R.P.', 'number': 'EUNF941023HTS...","ESQUEDA NIETO, Francisco Daniel","[ILLICIT-DRUGS-EO14059, SDGT]",(Linked To: CARTEL DEL NORESTE),https://ofac.treasury.gov/recent-actions/20250811,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[1994-10-23],NaN,"[Nuevo Laredo, Tamaulipas, Mexico]",[MX],"54505-ESQUEDA NIETO, Francisco Daniel","a.k.a. ESQUEDA, Franky;a.k.a. ESKEDA, Franki;a..."
22745,"[{'country': 'IR', 'address': None, 'city': No...","[ALIZADE HOUSHYAR, Javad, ALIZADEH-HOSHIAR, Ja...",54930,54930,"[{'type': 'National ID No.', 'number': '137656...","ALIZADEH HOSHYAR, Javad","[NPWMD, IFSR]",(Linked To: CONTROL AFZAR TABRIZ CO LTD),https://ofac.treasury.gov/recent-actions/20250811,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[1976-08-24],NaN,[Iran],[IR],"54930-ALIZADEH HOSHYAR, Javad",Arabic: جواد عليزاده هوشيار;a.k.a. ALIZADE HOU...
22755,"[{'city': 'Changzhou', 'country': 'CN', 'addre...",[CHANGZHOU JOMARS INDUSTRIAL AUTOMATION CO LTD...,54945,54945,"[{'type': 'Unified Social Credit Code (USCC)',...",CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,"[IFSR, NPWMD]",(Linked To: JOEMARS MACHINERY AND ELECTRIC IND...,https://ofac.treasury.gov/recent-actions/20250811,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[],NaN,[],[],54945-CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION ...,a.k.a. CHANGZHOU JOMARS INDUSTRIAL AUTOMATION ...


In [125]:
# droping duplicate cases based on same id-names
filtered_df2 = filtered_df.drop_duplicates(subset=['ID-NAME'], keep='last')
filtered_df2

,addresses,alt_names,entity_number,id,ids,name,programs,remarks,source,source_information_url,...,vessel_flag,vessel_owner,title,citizenships,dates_of_birth,country,places_of_birth,nationalities,ID-NAME,alias2
22572,"[{'city': 'Nuevo Laredo', 'country': 'MX', 'ad...","[ESQUEDA, Franky, Franky De La Joya, ESKEDA, F...",54505,54505,"[{'type': 'C.U.R.P.', 'number': 'EUNF941023HTS...","ESQUEDA NIETO, Francisco Daniel","[ILLICIT-DRUGS-EO14059, SDGT]",(Linked To: CARTEL DEL NORESTE),https://ofac.treasury.gov/recent-actions/20250811,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[1994-10-23],NaN,"[Nuevo Laredo, Tamaulipas, Mexico]",[MX],"54505-ESQUEDA NIETO, Francisco Daniel","a.k.a. ESQUEDA, Franky;a.k.a. ESKEDA, Franki;a..."
22745,"[{'country': 'IR', 'address': None, 'city': No...","[ALIZADE HOUSHYAR, Javad, ALIZADEH-HOSHIAR, Ja...",54930,54930,"[{'type': 'National ID No.', 'number': '137656...","ALIZADEH HOSHYAR, Javad","[NPWMD, IFSR]",(Linked To: CONTROL AFZAR TABRIZ CO LTD),https://ofac.treasury.gov/recent-actions/20250811,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[1976-08-24],NaN,[Iran],[IR],"54930-ALIZADEH HOSHYAR, Javad",Arabic: جواد عليزاده هوشيار;a.k.a. ALIZADE HOU...
22755,"[{'city': 'Changzhou', 'country': 'CN', 'addre...",[CHANGZHOU JOMARS INDUSTRIAL AUTOMATION CO LTD...,54945,54945,"[{'type': 'Unified Social Credit Code (USCC)',...",CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,"[IFSR, NPWMD]",(Linked To: JOEMARS MACHINERY AND ELECTRIC IND...,https://ofac.treasury.gov/recent-actions/20250811,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[],NaN,[],[],54945-CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION ...,a.k.a. CHANGZHOU JOMARS INDUSTRIAL AUTOMATION ...


In [126]:
filtered_df2.columns


Index(['addresses', 'alt_names', 'entity_number', 'id', 'ids', 'name',
       'programs', 'remarks', 'source', 'source_information_url',
       'source_list_url', 'type', 'federal_register_notice', 'start_date',
       'end_date', 'standard_order', 'license_policy', 'license_requirement',
       'call_sign', 'vessel_type', 'gross_tonnage', 'gross_registered_tonnage',
       'vessel_flag', 'vessel_owner', 'title', 'citizenships',
       'dates_of_birth', 'country', 'places_of_birth', 'nationalities',
       'ID-NAME', 'alias2'],
      dtype='object')

In [ ]:

# Convert alias2 strings into lists by splitting on ';', handle NaN safely
filtered_df2['alias2'] = filtered_df['alias2'].fillna('').apply(lambda x: [item.strip() for item in x.split(';') if item.strip()])


In [ ]:

# create a function to merge aliases from alter_name and aliases 2 keeping name with a.k.a and remove other without

def merge_alt_into_alias(row):
    alt_names = row['alt_names']  # plain names
    alias2 = row['alias2']        # may contain a.k.a. / f.k.a.

    # Create a set of stripped alias2 names for duplicate checking
    stripped_alias = set()
    for a in alias2:
        stripped = re.sub(r'^(a\.k\.a\.|f\.k\.a\.)\s*', '', a, flags=re.I).strip()
        stripped_alias.add(stripped)

    # Add alt_names only if not in stripped_alias
    for name in alt_names:
        if name not in stripped_alias:
            alias2.append(name)  # add the plain name

    return alias2

filtered_df2['alt_names'] = filtered_df2.apply(merge_alt_into_alias, axis=1)


# Drop alias2 column not needed now
filtered_df2.drop(columns=['alias2'], inplace=True)

In [129]:
filtered_df2['alt_names']

22572    [a.k.a. ESQUEDA, Franky, a.k.a. ESKEDA, Franki...
22745    [Arabic: جواد عليزاده هوشيار, a.k.a. ALIZADE H...
22755    [a.k.a. CHANGZHOU JOMARS INDUSTRIAL AUTOMATION...
Name: alt_names, dtype: object

In [130]:
filtered_df['alt_names'][22745]

['ALIZADE HOUSHYAR, Javad',
 'ALIZADEH-HOSHIAR, Javad',
 'ALIZADEH HUSHYAR, Javad',
 'ALIZADEH HOSHYAR, Javad Saeid']

In [131]:
filtered_df2['alt_names'][22745]

['Arabic: جواد عليزاده هوشيار',
 'a.k.a. ALIZADE HOUSHYAR, Javad',
 'a.k.a. ALIZADEH HOSHYAR, Javad Saeid',
 'a.k.a. ALIZADEH HUSHYAR, Javad',
 'a.k.a. ALIZADEH-HOSHIAR, Javad']

In [132]:
filtered_df2.columns

Index(['addresses', 'alt_names', 'entity_number', 'id', 'ids', 'name',
       'programs', 'remarks', 'source', 'source_information_url',
       'source_list_url', 'type', 'federal_register_notice', 'start_date',
       'end_date', 'standard_order', 'license_policy', 'license_requirement',
       'call_sign', 'vessel_type', 'gross_tonnage', 'gross_registered_tonnage',
       'vessel_flag', 'vessel_owner', 'title', 'citizenships',
       'dates_of_birth', 'country', 'places_of_birth', 'nationalities',
       'ID-NAME', 'alias2'],
      dtype='object')

In [32]:
temp_df = filtered_df2[['ID-NAME','entity_number', 'type','name','alt_names',
                        'places_of_birth','addresses','source', 'programs',                
                      'remarks','vessel_type','vessel_flag','vessel_owner',
                      'citizenships','dates_of_birth','country','nationalities','ids']]
temp_df['remarks'] = temp_df['remarks'].fillna('').str.split(';')
temp_df = temp_df.reset_index(drop=True)
temp_df.head()

C:\Users\wasi\AppData\Local\Temp\ipykernel_11036\2754430856.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['remarks'] = temp_df['remarks'].fillna('').str.split(';')


,ID-NAME,entity_number,type,name,alt_names,places_of_birth,addresses,source,programs,remarks,vessel_type,vessel_flag,vessel_owner,citizenships,dates_of_birth,country,nationalities,ids
0,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,"ESQUEDA NIETO, Francisco Daniel","[a.k.a. Franky De La Joya, Franky De La Joya, ...","[Nuevo Laredo, Tamaulipas, Mexico]","[{'city': 'Nuevo Laredo', 'country': 'MX', 'ad...",https://ofac.treasury.gov/recent-actions/20250811,"[ILLICIT-DRUGS-EO14059, SDGT]",[(Linked To: CARTEL DEL NORESTE)],None,None,None,[],[1994-10-23],NaN,[MX],"[{'type': 'C.U.R.P.', 'number': 'EUNF941023HTS..."
1,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","[ALIZADEH HOSHYAR, Javad Saeid, a.k.a. ALIZADE...",[Iran],"[{'country': 'IR', 'address': None, 'city': No...",https://ofac.treasury.gov/recent-actions/20250811,"[NPWMD, IFSR]",[(Linked To: CONTROL AFZAR TABRIZ CO LTD)],None,None,None,[],[1976-08-24],NaN,[IR],"[{'type': 'National ID No.', 'number': '137656..."
2,54945-CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION ...,54945,Entity,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,[CHANGZHOU JOMARS INDUSTRIAL AUTOMATION CO LTD...,[],"[{'city': 'Changzhou', 'country': 'CN', 'addre...",https://ofac.treasury.gov/recent-actions/20250811,"[IFSR, NPWMD]",[(Linked To: JOEMARS MACHINERY AND ELECTRIC IN...,None,None,None,[],[],NaN,[],"[{'type': 'Unified Social Credit Code (USCC)',..."


In [ ]:

temp_df.columns


Index(['ID-NAME', 'entity_number', 'type', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country', 'nationalities', 'ids'],
      dtype='object')

In [33]:
type(temp_df.ids[0])

list

In [34]:
type(temp_df['remarks'].iloc[0])

list

In [35]:



# extracting sanction date from source link
temp_df['sanction_date'] = temp_df['source'].str.rsplit('/', n=1).str[-1].str[:8]

temp_df

,ID-NAME,entity_number,type,name,alt_names,places_of_birth,addresses,source,programs,remarks,vessel_type,vessel_flag,vessel_owner,citizenships,dates_of_birth,country,nationalities,ids,sanction_date
0,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,"ESQUEDA NIETO, Francisco Daniel","[a.k.a. Franky De La Joya, Franky De La Joya, ...","[Nuevo Laredo, Tamaulipas, Mexico]","[{'city': 'Nuevo Laredo', 'country': 'MX', 'ad...",https://ofac.treasury.gov/recent-actions/20250811,"[ILLICIT-DRUGS-EO14059, SDGT]",[(Linked To: CARTEL DEL NORESTE)],None,None,None,[],[1994-10-23],NaN,[MX],"[{'type': 'C.U.R.P.', 'number': 'EUNF941023HTS...",20250811
1,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","[ALIZADEH HOSHYAR, Javad Saeid, a.k.a. ALIZADE...",[Iran],"[{'country': 'IR', 'address': None, 'city': No...",https://ofac.treasury.gov/recent-actions/20250811,"[NPWMD, IFSR]",[(Linked To: CONTROL AFZAR TABRIZ CO LTD)],None,None,None,[],[1976-08-24],NaN,[IR],"[{'type': 'National ID No.', 'number': '137656...",20250811
2,54945-CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION ...,54945,Entity,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,[CHANGZHOU JOMARS INDUSTRIAL AUTOMATION CO LTD...,[],"[{'city': 'Changzhou', 'country': 'CN', 'addre...",https://ofac.treasury.gov/recent-actions/20250811,"[IFSR, NPWMD]",[(Linked To: JOEMARS MACHINERY AND ELECTRIC IN...,None,None,None,[],[],NaN,[],"[{'type': 'Unified Social Credit Code (USCC)',...",20250811


In [36]:
# extracting order_id from program name if available
import re
# function to extract from program name if found
def extract_order_ids(program_list):
    pattern = r'EO\d+'
    matches = []
    for item in program_list:
        if item:
            matches.extend(re.findall(pattern, item))
    # Replace 'EO' with 'E.O.' in all matched strings
    matches = [m.replace('EO', 'E.O.') for m in matches]
    return "; ".join(matches)

temp_df['order_id'] = temp_df['programs'].apply(extract_order_ids)

In [37]:
# joining all program separated by ;
temp_df['programs'] = temp_df['programs'].apply(
    lambda x: '; '.join(str(i) for i in x if i is not None) if isinstance(x, list) else x
)

In [38]:
temp_df['programs']

0    ILLICIT-DRUGS-EO14059; SDGT
1                    NPWMD; IFSR
2                    IFSR; NPWMD
Name: programs, dtype: object

In [39]:
temp_df.columns

Index(['ID-NAME', 'entity_number', 'type', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country', 'nationalities', 'ids', 'sanction_date',
       'order_id'],
      dtype='object')

In [40]:
#convert string list to actual list
import ast

def parse_ids_column(val):
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except (ValueError, SyntaxError):
            return []
    return val

temp_df['ids'] = temp_df['ids'].apply(parse_ids_column)

In [41]:
# this function extract all usefull values from ids into seperate column and extract all E.O. form ids key and type key
import pandas as pd
import re

wanted_types = [
    'Phone Number',
    'Target Type',
    'Website',
    'Organization Established Date',
    'Organization Type:',
    'Email Address',
    'Gender'
]

eo_patterns = [
    'Executive Order 14024',
    'Executive Order 13662',
    'Executive Order 13846'
]

remove_types = [
    'Additional Sanctions Information -',
    'Effective Date (CMIC)',
    'Effective Date (EO 14024 Directive 1a):',
    'Effective Date (EO 14024 Directive 3):',
    'Effective Date (EO 14024 Directive 2):',
    'Secondary sanctions risk:',
    'Transactions Prohibited For Persons Owned or Controlled By U.S. Financial Institutions:',
    'Issuer Name'
] + wanted_types  # Also remove extracted wanted types from ids

# Prepare a normalized list for comparison (lowercase, stripped, no trailing colon)
remove_types_lower = [t.lower().strip().rstrip(':') for t in remove_types]

def process_ids(row):
    ids_list = row.get('ids')
    if not isinstance(ids_list, list):
        # If ids_list is not a list, return None/NaN for all extracted fields
        data = {f"{k}_extracted": None for k in wanted_types}
        data['order_id_extracted'] = row.get('order_id', None)
        data['ids_extracted'] = ids_list
        return pd.Series(data)

    extracted_values = {f"{k}_extracted": None for k in wanted_types}
    order_ids = []

    for d in ids_list:
        t = d.get('type')
        n = d.get('number')

        # Extract wanted types values
        if t and t in wanted_types:
            extracted_values[f"{t}_extracted"] = n

        # Extract EO codes from type field
        if t and any(pattern in t for pattern in eo_patterns):
            match = re.search(r'Executive Order (\d+)', t)
            if match:
                eo_code = f"E.O.{match.group(1)}"
                order_ids.append(eo_code)

        # NEW: Extract EO codes from number field (string)
        if n and isinstance(n, str):
            matches = re.findall(r'Executive Order (\d+)', n)
            for m in matches:
                eo_code = f"E.O.{m}"
                order_ids.append(eo_code)

    # Combine existing order_id with extracted EO codes, remove duplicates
    existing_order_id = row.get('order_id')
    all_order_ids = [existing_order_id] if existing_order_id else []
    all_order_ids.extend(order_ids)
    # Remove duplicates and join with "; "
    combined_order_id = "; ".join(sorted(set(filter(None, all_order_ids)))) if all_order_ids else None

    # Filter out unwanted 'type's robustly (ignore case, strip spaces and trailing colon)
    filtered_ids = [
        d for d in ids_list
        if d.get('type') is None or d.get('type').lower().strip().rstrip(':') not in remove_types_lower
    ]

    extracted_values['order_id_extracted'] = combined_order_id
    extracted_values['ids_extracted'] = filtered_ids
    return pd.Series(extracted_values)

# Usage example:

temp_df = temp_df.join(temp_df.apply(process_ids, axis=1))


In [42]:
temp_df.columns

Index(['ID-NAME', 'entity_number', 'type', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country', 'nationalities', 'ids', 'sanction_date',
       'order_id', 'Phone Number_extracted', 'Target Type_extracted',
       'Website_extracted', 'Organization Established Date_extracted',
       'Organization Type:_extracted', 'Email Address_extracted',
       'Gender_extracted', 'order_id_extracted', 'ids_extracted'],
      dtype='object')

In [43]:
temp_df = temp_df.drop(columns=['ids','order_id'])

In [44]:
temp_df.columns

Index(['ID-NAME', 'entity_number', 'type', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country', 'nationalities', 'sanction_date',
       'Phone Number_extracted', 'Target Type_extracted', 'Website_extracted',
       'Organization Established Date_extracted',
       'Organization Type:_extracted', 'Email Address_extracted',
       'Gender_extracted', 'order_id_extracted', 'ids_extracted'],
      dtype='object')

In [45]:
# add fixed columns  
temp_df['source_description'] = 'United States Department of the Treasury'
temp_df['sanction_authority'] = 'U.S. Department of Treasury - Office of Foreign Assets Control'
temp_df['sanction_list_name'] = 'Specially Designated Nationals (SDN) - Treasury Department'
temp_df['sanction_authority_ids'] = '19'
temp_df['sanction_authority_country'] = 'USA'
temp_df['subject_country'] = 'United States'
temp_df['source_type'] = 'Sanction'
temp_df['is_pep'] = 'False'
temp_df['is_sanction'] = 'True'
temp_df['is_watchlist'] = 'False'
temp_df['is_enforcement'] = 'False'
temp_df['is_apc'] = 'False'


In [46]:
temp_df

,ID-NAME,entity_number,type,name,alt_names,places_of_birth,addresses,source,programs,remarks,...,sanction_list_name,sanction_authority_ids,sanction_authority_country,subject_country,source_type,is_pep,is_sanction,is_watchlist,is_enforcement,is_apc
0,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,"ESQUEDA NIETO, Francisco Daniel","[a.k.a. Franky De La Joya, Franky De La Joya, ...","[Nuevo Laredo, Tamaulipas, Mexico]","[{'city': 'Nuevo Laredo', 'country': 'MX', 'ad...",https://ofac.treasury.gov/recent-actions/20250811,ILLICIT-DRUGS-EO14059; SDGT,[(Linked To: CARTEL DEL NORESTE)],...,Specially Designated Nationals (SDN) - Treasur...,19,USA,United States,Sanction,False,True,False,False,False
1,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","[ALIZADEH HOSHYAR, Javad Saeid, a.k.a. ALIZADE...",[Iran],"[{'country': 'IR', 'address': None, 'city': No...",https://ofac.treasury.gov/recent-actions/20250811,NPWMD; IFSR,[(Linked To: CONTROL AFZAR TABRIZ CO LTD)],...,Specially Designated Nationals (SDN) - Treasur...,19,USA,United States,Sanction,False,True,False,False,False
2,54945-CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION ...,54945,Entity,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,[CHANGZHOU JOMARS INDUSTRIAL AUTOMATION CO LTD...,[],"[{'city': 'Changzhou', 'country': 'CN', 'addre...",https://ofac.treasury.gov/recent-actions/20250811,IFSR; NPWMD,[(Linked To: JOEMARS MACHINERY AND ELECTRIC IN...,...,Specially Designated Nationals (SDN) - Treasur...,19,USA,United States,Sanction,False,True,False,False,False


In [47]:
temp_df.columns

Index(['ID-NAME', 'entity_number', 'type', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country', 'nationalities', 'sanction_date',
       'Phone Number_extracted', 'Target Type_extracted', 'Website_extracted',
       'Organization Established Date_extracted',
       'Organization Type:_extracted', 'Email Address_extracted',
       'Gender_extracted', 'order_id_extracted', 'ids_extracted',
       'source_description', 'sanction_authority', 'sanction_list_name',
       'sanction_authority_ids', 'sanction_authority_country',
       'subject_country', 'source_type', 'is_pep', 'is_sanction',
       'is_watchlist', 'is_enforcement', 'is_apc'],
      dtype='object')

In [48]:
# Ensure all columns have lists and replace NaN or other non-list types with empty lists

columns_to_process = ['name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country', 'nationalities', 'sanction_date'
       , 'Phone Number_extracted', 'Target Type_extracted',
       'Website_extracted', 'Organization Established Date_extracted',
       'Organization Type:_extracted', 'Email Address_extracted',
       'Gender_extracted', 'order_id_extracted', 'ids_extracted',
       'source_description', 'sanction_authority', 'sanction_list_name',
       'sanction_authority_ids', 'sanction_authority_country',
       'subject_country', 'source_type', 'is_pep', 'is_sanction',
       'is_watchlist', 'is_enforcement', 'is_apc'
                      ]

for col in columns_to_process:
    temp_df[col] = temp_df[col].apply(
        lambda x: x if isinstance(x, list) else [str(x)] if pd.notnull(x) else []
    )

# Step 1: Get the max length of lists across the columns
max_len = temp_df[columns_to_process].applymap(len).max(axis=1)

# Step 2: Pad the shorter lists with None
for col in columns_to_process:
    temp_df[col] = temp_df.apply(lambda row: row[col] + [None] * (max_len[row.name] - len(row[col])), axis=1)

# Step 3: Explode the lists into separate rows
temp_df_expanded = temp_df.explode(columns_to_process, ignore_index=True)

# Display the expanded DataFrame
temp_df_expanded


C:\Users\wasi\AppData\Local\Temp\ipykernel_11036\3165593618.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  max_len = temp_df[columns_to_process].applymap(len).max(axis=1)


,ID-NAME,entity_number,type,name,alt_names,places_of_birth,addresses,source,programs,remarks,...,sanction_list_name,sanction_authority_ids,sanction_authority_country,subject_country,source_type,is_pep,is_sanction,is_watchlist,is_enforcement,is_apc
0,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,"ESQUEDA NIETO, Francisco Daniel",a.k.a. Franky De La Joya,"Nuevo Laredo, Tamaulipas, Mexico","{'city': 'Nuevo Laredo', 'country': 'MX', 'add...",https://ofac.treasury.gov/recent-actions/20250811,ILLICIT-DRUGS-EO14059; SDGT,(Linked To: CARTEL DEL NORESTE),...,Specially Designated Nationals (SDN) - Treasur...,19,USA,United States,Sanction,False,True,False,False,False
1,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,Franky De La Joya,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"ESKEDA, Franki",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"a.k.a. ESQUEDA, Franky",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"a.k.a. ESKEDA, Franki",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"ESQUEDA, Franky",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","ALIZADEH HOSHYAR, Javad Saeid",Iran,"{'country': 'IR', 'address': None, 'city': Non...",https://ofac.treasury.gov/recent-actions/20250811,NPWMD; IFSR,(Linked To: CONTROL AFZAR TABRIZ CO LTD),...,Specially Designated Nationals (SDN) - Treasur...,19,USA,United States,Sanction,False,True,False,False,False
7,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADEH HUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADEH HOSHYAR, Javad Saeid",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADE HOUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [49]:

temp_df_expanded

,ID-NAME,entity_number,type,name,alt_names,places_of_birth,addresses,source,programs,remarks,...,sanction_list_name,sanction_authority_ids,sanction_authority_country,subject_country,source_type,is_pep,is_sanction,is_watchlist,is_enforcement,is_apc
0,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,"ESQUEDA NIETO, Francisco Daniel",a.k.a. Franky De La Joya,"Nuevo Laredo, Tamaulipas, Mexico","{'city': 'Nuevo Laredo', 'country': 'MX', 'add...",https://ofac.treasury.gov/recent-actions/20250811,ILLICIT-DRUGS-EO14059; SDGT,(Linked To: CARTEL DEL NORESTE),...,Specially Designated Nationals (SDN) - Treasur...,19,USA,United States,Sanction,False,True,False,False,False
1,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,Franky De La Joya,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"ESKEDA, Franki",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"a.k.a. ESQUEDA, Franky",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"a.k.a. ESKEDA, Franki",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"ESQUEDA, Franky",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","ALIZADEH HOSHYAR, Javad Saeid",Iran,"{'country': 'IR', 'address': None, 'city': Non...",https://ofac.treasury.gov/recent-actions/20250811,NPWMD; IFSR,(Linked To: CONTROL AFZAR TABRIZ CO LTD),...,Specially Designated Nationals (SDN) - Treasur...,19,USA,United States,Sanction,False,True,False,False,False
7,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADEH HUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADEH HOSHYAR, Javad Saeid",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADE HOUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [50]:
temp_df_expanded.columns

Index(['ID-NAME', 'entity_number', 'type', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country', 'nationalities', 'sanction_date',
       'Phone Number_extracted', 'Target Type_extracted', 'Website_extracted',
       'Organization Established Date_extracted',
       'Organization Type:_extracted', 'Email Address_extracted',
       'Gender_extracted', 'order_id_extracted', 'ids_extracted',
       'source_description', 'sanction_authority', 'sanction_list_name',
       'sanction_authority_ids', 'sanction_authority_country',
       'subject_country', 'source_type', 'is_pep', 'is_sanction',
       'is_watchlist', 'is_enforcement', 'is_apc'],
      dtype='object')

In [51]:
# Renaming country and type column to solve the naming confilict with country and type in address dictionary expanding into new columns  

temp_df_expanded.rename(columns={'country': 'country1', 'type': 'type1'}, inplace=True)
temp_df_expanded.columns

Index(['ID-NAME', 'entity_number', 'type1', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country1', 'nationalities', 'sanction_date',
       'Phone Number_extracted', 'Target Type_extracted', 'Website_extracted',
       'Organization Established Date_extracted',
       'Organization Type:_extracted', 'Email Address_extracted',
       'Gender_extracted', 'order_id_extracted', 'ids_extracted',
       'source_description', 'sanction_authority', 'sanction_list_name',
       'sanction_authority_ids', 'sanction_authority_country',
       'subject_country', 'source_type', 'is_pep', 'is_sanction',
       'is_watchlist', 'is_enforcement', 'is_apc'],
      dtype='object')

In [52]:
temp_df_expanded.addresses.loc[0]

{'city': 'Nuevo Laredo',
 'country': 'MX',
 'address': None,
 'state': 'Tamaulipas',
 'postal_code': None}

In [53]:

# Handle None values and apply pd.Series to expand the dictionary into new seperate columns here "addresses" and "ids" are dictionary columns

temp_df_expanded1 = temp_df_expanded.join(temp_df_expanded['addresses'].apply(lambda x: pd.Series(x) if isinstance(x, dict) else pd.Series({})))


In [54]:
# Renaming the country name which we get from expanding address dictionary to solve the naming confilict with country in ids dictionary 
temp_df_expanded1 = temp_df_expanded1.rename(columns={'country': 'address_country'})

In [55]:
temp_df_expanded1.columns

Index(['ID-NAME', 'entity_number', 'type1', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country1', 'nationalities', 'sanction_date',
       'Phone Number_extracted', 'Target Type_extracted', 'Website_extracted',
       'Organization Established Date_extracted',
       'Organization Type:_extracted', 'Email Address_extracted',
       'Gender_extracted', 'order_id_extracted', 'ids_extracted',
       'source_description', 'sanction_authority', 'sanction_list_name',
       'sanction_authority_ids', 'sanction_authority_country',
       'subject_country', 'source_type', 'is_pep', 'is_sanction',
       'is_watchlist', 'is_enforcement', 'is_apc', 'city', 'address_country',
       'address', 'state', 'postal_code'],
      dtype='object')

In [56]:
# expanding ids column dictionary into separate columns 
temp_df_expanded1 = temp_df_expanded1.join(temp_df_expanded['ids_extracted'].apply(lambda x: pd.Series(x) if isinstance(x, dict) else pd.Series({})))
temp_df_expanded1

,ID-NAME,entity_number,type1,name,alt_names,places_of_birth,addresses,source,programs,remarks,...,city,address_country,address,state,postal_code,type,number,country,issue_date,expiration_date
0,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,"ESQUEDA NIETO, Francisco Daniel",a.k.a. Franky De La Joya,"Nuevo Laredo, Tamaulipas, Mexico","{'city': 'Nuevo Laredo', 'country': 'MX', 'add...",https://ofac.treasury.gov/recent-actions/20250811,ILLICIT-DRUGS-EO14059; SDGT,(Linked To: CARTEL DEL NORESTE),...,Nuevo Laredo,MX,None,Tamaulipas,NaN,C.U.R.P.,EUNF941023HTSSTR02,MX,NaN,NaN
1,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,Franky De La Joya,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"ESKEDA, Franki",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"a.k.a. ESQUEDA, Franky",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"a.k.a. ESKEDA, Franki",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"ESQUEDA, Franky",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","ALIZADEH HOSHYAR, Javad Saeid",Iran,"{'country': 'IR', 'address': None, 'city': Non...",https://ofac.treasury.gov/recent-actions/20250811,NPWMD; IFSR,(Linked To: CONTROL AFZAR TABRIZ CO LTD),...,None,IR,None,None,NaN,National ID No.,1376561999,IR,NaN,NaN
7,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADEH HUSHYAR, Javad",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADEH HOSHYAR, Javad Saeid",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADE HOUSHYAR, Javad",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
temp_df_expanded1.columns


Index(['ID-NAME', 'entity_number', 'type1', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country1', 'nationalities', 'sanction_date',
       'Phone Number_extracted', 'Target Type_extracted', 'Website_extracted',
       'Organization Established Date_extracted',
       'Organization Type:_extracted', 'Email Address_extracted',
       'Gender_extracted', 'order_id_extracted', 'ids_extracted',
       'source_description', 'sanction_authority', 'sanction_list_name',
       'sanction_authority_ids', 'sanction_authority_country',
       'subject_country', 'source_type', 'is_pep', 'is_sanction',
       'is_watchlist', 'is_enforcement', 'is_apc', 'city', 'address_country',
       'address', 'state', 'postal_code', 'type', 'number', 'country',
       'issue_date', 'expiration_date'],
      dtype='object')

In [58]:
temp_df_expanded1.address_country

0      MX
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6      IR
7     NaN
8     NaN
9     NaN
10    NaN
11    NaN
12    NaN
13    NaN
14    NaN
15     CN
16    NaN
17    NaN
18    NaN
19    NaN
Name: address_country, dtype: object

In [59]:
temp_df_expanded1.columns

Index(['ID-NAME', 'entity_number', 'type1', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country1', 'nationalities', 'sanction_date',
       'Phone Number_extracted', 'Target Type_extracted', 'Website_extracted',
       'Organization Established Date_extracted',
       'Organization Type:_extracted', 'Email Address_extracted',
       'Gender_extracted', 'order_id_extracted', 'ids_extracted',
       'source_description', 'sanction_authority', 'sanction_list_name',
       'sanction_authority_ids', 'sanction_authority_country',
       'subject_country', 'source_type', 'is_pep', 'is_sanction',
       'is_watchlist', 'is_enforcement', 'is_apc', 'city', 'address_country',
       'address', 'state', 'postal_code', 'type', 'number', 'country',
       'issue_date', 'expiration_date'],
      dtype='object')

In [60]:
temp_df_expanded2 = temp_df_expanded1.rename(columns={'country': 'identifier_country',
                                  'address': 'address_street',
                                  'city':'address_city',
                                  'state':'address_province',
                                 'postal_code':'address_postal_code',
                                  'type':'identifier_name',
                                 'number':'identifier_value',
                                 'issue_date':'identifier_issue_date_date',
                                 'expiration_date':'identifier_expiry_date_date',
                                 'citizenships':'citizenship_country',
                                 'nationalities':'nationality_country',
                                 'alt_names':'alias_name',
                                 'type1':'type',
                                 'programs':'sanction_programme_name',
                                 'remarks':'position',
                                 'vessel_type':'vessel_type',
                                 'vessel_flag':'current_country_flag',
                                 'sanction_date':'sanction_action_date_date',
                                 'Phone Number_extracted':'contact_number',
                                 'Website_extracted':'website',
                                 'Organization Established Date_extracted':'date_of_incorporation_date',
                                 'Organization Type:_extracted':'sanction_remarks',
                                 'Email Address_extracted':'email_id',
                                 'Gender_extracted':'gender',
                                 'order_id_extracted':'sanction_order_number',
                                 'entity_number':'ID',
                                
                                 })

temp_df_expanded2['type'] = temp_df_expanded2['type'].replace('Entity','Organization')

temp_df_expanded2

,ID-NAME,ID,type,name,alias_name,places_of_birth,addresses,source,sanction_programme_name,position,...,address_city,address_country,address_street,address_province,address_postal_code,identifier_name,identifier_value,identifier_country,identifier_issue_date_date,identifier_expiry_date_date
0,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,"ESQUEDA NIETO, Francisco Daniel",a.k.a. Franky De La Joya,"Nuevo Laredo, Tamaulipas, Mexico","{'city': 'Nuevo Laredo', 'country': 'MX', 'add...",https://ofac.treasury.gov/recent-actions/20250811,ILLICIT-DRUGS-EO14059; SDGT,(Linked To: CARTEL DEL NORESTE),...,Nuevo Laredo,MX,None,Tamaulipas,NaN,C.U.R.P.,EUNF941023HTSSTR02,MX,NaN,NaN
1,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,Franky De La Joya,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"ESKEDA, Franki",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"a.k.a. ESQUEDA, Franky",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"a.k.a. ESKEDA, Franki",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"ESQUEDA, Franky",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","ALIZADEH HOSHYAR, Javad Saeid",Iran,"{'country': 'IR', 'address': None, 'city': Non...",https://ofac.treasury.gov/recent-actions/20250811,NPWMD; IFSR,(Linked To: CONTROL AFZAR TABRIZ CO LTD),...,None,IR,None,None,NaN,National ID No.,1376561999,IR,NaN,NaN
7,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADEH HUSHYAR, Javad",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADEH HOSHYAR, Javad Saeid",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADE HOUSHYAR, Javad",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
# spiliting birth address into different columns
import numpy as np
def split_pob_with_type(value):
    if pd.isna(value):
        return pd.Series([pd.NA, pd.NA, pd.NA])
    address_type = 'Birth'
    if ',' in value:
        parts = value.rsplit(',', 1)
        return pd.Series([address_type, parts[0].strip(), parts[1].strip()])
    else:
        return pd.Series([address_type, pd.NA, value.strip()])

temp_df_expanded2[['pob_address_type', 'pob_address', 'pob_country']] = temp_df_expanded2['places_of_birth'].apply(split_pob_with_type)
temp_df_expanded2['address_type']= np.nan

In [62]:
temp_df_expanded2.pob_country

0     Mexico
1       <NA>
2       <NA>
3       <NA>
4       <NA>
5       <NA>
6       Iran
7       <NA>
8       <NA>
9       <NA>
10      <NA>
11      <NA>
12      <NA>
13      <NA>
14      <NA>
15      <NA>
16      <NA>
17      <NA>
18      <NA>
19      <NA>
Name: pob_country, dtype: object

In [63]:
temp_df_expanded2.columns


Index(['ID-NAME', 'ID', 'type', 'name', 'alias_name', 'places_of_birth',
       'addresses', 'source', 'sanction_programme_name', 'position',
       'vessel_type', 'current_country_flag', 'vessel_owner',
       'citizenship_country', 'dates_of_birth', 'country1',
       'nationality_country', 'sanction_action_date_date', 'contact_number',
       'Target Type_extracted', 'website', 'date_of_incorporation_date',
       'sanction_remarks', 'email_id', 'gender', 'sanction_order_number',
       'ids_extracted', 'source_description', 'sanction_authority',
       'sanction_list_name', 'sanction_authority_ids',
       'sanction_authority_country', 'subject_country', 'source_type',
       'is_pep', 'is_sanction', 'is_watchlist', 'is_enforcement', 'is_apc',
       'address_city', 'address_country', 'address_street', 'address_province',
       'address_postal_code', 'identifier_name', 'identifier_value',
       'identifier_country', 'identifier_issue_date_date',
       'identifier_expiry_date_dat

In [64]:
# importing countrycode df
country_code_df = pd.read_excel(r"Country_code.xlsx",
                   keep_default_na=False, na_values=[])
country_code_df


,record,Country1,Country,countrycode3,countrycode2
0,,Afghanistan,Afghanistan,AFG,AF
1,,Aland Islands,Aland Islands,ALA,AX
2,,Albania,Albania,ALB,AL
3,,Algeria,Algeria,DZA,DZ
4,,American Samoa,American Samoa,ASM,AS
...,...,...,...,...,...
234,,Wallis and Futuna,Wallis and Futuna,WLF,WF
235,,Western Sahara,Western Sahara,ESH,EH
236,,Yemen,Yemen,YEM,YE
237,,Zambia,Zambia,ZMB,ZM


In [65]:

# Create a mapping dictionary from country_code_df
code_to_name = dict(zip(country_code_df['countrycode2'], country_code_df['Country']))

# Replace country codes with country names in the specified columns of temp_df_expanded2
columns_to_map = ['address_country', 'nationality_country', 'citizenship_country','identifier_country']
for col in columns_to_map:
    temp_df_expanded2[col] = temp_df_expanded2[col].map(code_to_name).fillna(temp_df_expanded2[col])

# Create mapping dictionaries for countrycode3
code_to_countrycode3 = dict(zip(country_code_df['Country'], country_code_df['countrycode3']))

# Add new columns for 'countrycode3' based on 'country', 'nationality','pob_country' and 'citizen'
columns_to_map2 = ['address_country', 'nationality_country', 'citizenship_country','identifier_country','pob_country']
for col in columns_to_map2:
    temp_df_expanded2[f'{col}_code'] = temp_df_expanded2[col].map(code_to_countrycode3).fillna(temp_df_expanded2[col])

    
    
temp_df_expanded2.pob_country

0     Mexico
1       <NA>
2       <NA>
3       <NA>
4       <NA>
5       <NA>
6       Iran
7       <NA>
8       <NA>
9       <NA>
10      <NA>
11      <NA>
12      <NA>
13      <NA>
14      <NA>
15      <NA>
16      <NA>
17      <NA>
18      <NA>
19      <NA>
Name: pob_country, dtype: object

In [66]:
temp_df_expanded2.columns

Index(['ID-NAME', 'ID', 'type', 'name', 'alias_name', 'places_of_birth',
       'addresses', 'source', 'sanction_programme_name', 'position',
       'vessel_type', 'current_country_flag', 'vessel_owner',
       'citizenship_country', 'dates_of_birth', 'country1',
       'nationality_country', 'sanction_action_date_date', 'contact_number',
       'Target Type_extracted', 'website', 'date_of_incorporation_date',
       'sanction_remarks', 'email_id', 'gender', 'sanction_order_number',
       'ids_extracted', 'source_description', 'sanction_authority',
       'sanction_list_name', 'sanction_authority_ids',
       'sanction_authority_country', 'subject_country', 'source_type',
       'is_pep', 'is_sanction', 'is_watchlist', 'is_enforcement', 'is_apc',
       'address_city', 'address_country', 'address_street', 'address_province',
       'address_postal_code', 'identifier_name', 'identifier_value',
       'identifier_country', 'identifier_issue_date_date',
       'identifier_expiry_date_dat

In [67]:
# spliting the sanction date

temp_df_expanded2['sanction_action_date_month'] = temp_df_expanded2['sanction_action_date_date'].str[4:6]       # 5th and 6th characters
temp_df_expanded2['sanction_action_date_year'] = temp_df_expanded2['sanction_action_date_date'].str[:4]     # Left 4 characters
temp_df_expanded2['sanction_action_date_date'] = temp_df_expanded2['sanction_action_date_date'].str[6:8]       # 7th and 8th characters

In [68]:
# selecting important columns except pob address and rearranging them in order
temp_df_expanded3 = temp_df_expanded2[['ID-NAME', 'ID', 'type', 'name', 'alias_name',
                                       'gender','dates_of_birth','citizenship_country','citizenship_country_code',
                                       'nationality_country','nationality_country_code','date_of_incorporation_date',
                                       'vessel_type','current_country_flag','address_type',
                                       'address_street','address_city', 'address_province', 'address_postal_code',
                                       'address_country','address_country_code','contact_number','email_id',
                                       'website','identifier_name','identifier_value','identifier_country',
                                       'identifier_country_code','identifier_issue_date_date','identifier_expiry_date_date',
                                       'position','sanction_authority','sanction_authority_ids','sanction_authority_country',
                                       'sanction_list_name','sanction_action_date_date','sanction_action_date_month',
                                       'sanction_action_date_year','sanction_order_number',
                                       'sanction_programme_name','source_type','source','source_description',
                                       'is_pep', 'is_sanction', 'is_watchlist', 'is_enforcement', 'is_apc',
                                       'subject_country'
                                        ]]


temp_df_expanded3

,ID-NAME,ID,type,name,alias_name,gender,dates_of_birth,citizenship_country,citizenship_country_code,nationality_country,...,sanction_programme_name,source_type,source,source_description,is_pep,is_sanction,is_watchlist,is_enforcement,is_apc,subject_country
0,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,"ESQUEDA NIETO, Francisco Daniel",a.k.a. Franky De La Joya,Male,1994-10-23,None,None,Mexico,...,ILLICIT-DRUGS-EO14059; SDGT,Sanction,https://ofac.treasury.gov/recent-actions/20250811,United States Department of the Treasury,False,True,False,False,False,United States
1,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,Franky De La Joya,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"ESKEDA, Franki",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"a.k.a. ESQUEDA, Franky",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"a.k.a. ESKEDA, Franki",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"ESQUEDA, Franky",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","ALIZADEH HOSHYAR, Javad Saeid",Male,1976-08-24,None,None,Iran,...,NPWMD; IFSR,Sanction,https://ofac.treasury.gov/recent-actions/20250811,United States Department of the Treasury,False,True,False,False,False,United States
7,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADEH HUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADEH HOSHYAR, Javad Saeid",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADE HOUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [69]:
# selecting seperate pob address columns and not nan values
pob_df = temp_df_expanded2[['ID-NAME','ID','type','pob_address_type', 'pob_address','pob_country', 'pob_country_code']]
pob_df = pob_df[pob_df['pob_country'].notna()]
pob_df

,ID-NAME,ID,type,pob_address_type,pob_address,pob_country,pob_country_code
0,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,Birth,"Nuevo Laredo, Tamaulipas",Mexico,MEX
6,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,Birth,<NA>,Iran,IRN


In [70]:
pob_df = pob_df.rename(columns={
    'pob_address_type': 'address_type',
    'pob_address': 'address_street',
    'pob_country': 'address_country',
    'pob_country_code': 'address_country_code'
})
pob_df

,ID-NAME,ID,type,address_type,address_street,address_country,address_country_code
0,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,Birth,"Nuevo Laredo, Tamaulipas",Mexico,MEX
6,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,Birth,<NA>,Iran,IRN


In [71]:
# adding pob_df value below temp_df_expanded3
temp_df_expanded4 = pd.concat([temp_df_expanded3, pob_df], ignore_index=True)
# sort based on ID column and 
temp_df_expanded4 = temp_df_expanded4.sort_values(by=['ID', 'name'],na_position='last').reset_index(drop=True)
temp_df_expanded4

,ID-NAME,ID,type,name,alias_name,gender,dates_of_birth,citizenship_country,citizenship_country_code,nationality_country,...,sanction_programme_name,source_type,source,source_description,is_pep,is_sanction,is_watchlist,is_enforcement,is_apc,subject_country
0,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,"ESQUEDA NIETO, Francisco Daniel",a.k.a. Franky De La Joya,Male,1994-10-23,None,None,Mexico,...,ILLICIT-DRUGS-EO14059; SDGT,Sanction,https://ofac.treasury.gov/recent-actions/20250811,United States Department of the Treasury,False,True,False,False,False,United States
1,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,Franky De La Joya,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"ESKEDA, Franki",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"a.k.a. ESQUEDA, Franky",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"a.k.a. ESKEDA, Franki",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"ESQUEDA, Franky",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","ALIZADEH HOSHYAR, Javad Saeid",Male,1976-08-24,None,None,Iran,...,NPWMD; IFSR,Sanction,https://ofac.treasury.gov/recent-actions/20250811,United States Department of the Treasury,False,True,False,False,False,United States
8,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADEH HUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADEH HOSHYAR, Javad Saeid",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [72]:
temp_df_expanded4

,ID-NAME,ID,type,name,alias_name,gender,dates_of_birth,citizenship_country,citizenship_country_code,nationality_country,...,sanction_programme_name,source_type,source,source_description,is_pep,is_sanction,is_watchlist,is_enforcement,is_apc,subject_country
0,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,"ESQUEDA NIETO, Francisco Daniel",a.k.a. Franky De La Joya,Male,1994-10-23,None,None,Mexico,...,ILLICIT-DRUGS-EO14059; SDGT,Sanction,https://ofac.treasury.gov/recent-actions/20250811,United States Department of the Treasury,False,True,False,False,False,United States
1,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,Franky De La Joya,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"ESKEDA, Franki",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"a.k.a. ESQUEDA, Franky",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"a.k.a. ESKEDA, Franki",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"ESQUEDA, Franky",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","ALIZADEH HOSHYAR, Javad Saeid",Male,1976-08-24,None,None,Iran,...,NPWMD; IFSR,Sanction,https://ofac.treasury.gov/recent-actions/20250811,United States Department of the Treasury,False,True,False,False,False,United States
8,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADEH HUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADEH HOSHYAR, Javad Saeid",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [73]:
# extracting the column whic need to be sorted from main temp_df_expanded4
sort_df = temp_df_expanded4[['ID-NAME','ID','type','address_type','address_street','address_city','address_province','address_postal_code','address_country','address_country_code']]
# Sort by entity_number ascending and address_country with NaNs last
df_sorted = sort_df.sort_values(
    by=['ID', 'address_country'],
    na_position='last'
).reset_index(drop=True)
df_sorted = df_sorted[['address_type','address_street','address_city','address_province','address_postal_code','address_country','address_country_code']]
df_sorted

,address_type,address_street,address_city,address_province,address_postal_code,address_country,address_country_code
0,NaN,None,Nuevo Laredo,Tamaulipas,NaN,Mexico,MEX
1,Birth,"Nuevo Laredo, Tamaulipas",NaN,NaN,NaN,Mexico,MEX
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,None,None,None,NaN,Iran,IRN
8,Birth,<NA>,NaN,NaN,NaN,Iran,IRN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
# removing unsorted address columns from temp_df_expanded4
temp_df_expanded4 = temp_df_expanded4.drop(df_sorted.columns, axis=1)
temp_df_expanded4

,ID-NAME,ID,type,name,alias_name,gender,dates_of_birth,citizenship_country,citizenship_country_code,nationality_country,...,sanction_programme_name,source_type,source,source_description,is_pep,is_sanction,is_watchlist,is_enforcement,is_apc,subject_country
0,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,"ESQUEDA NIETO, Francisco Daniel",a.k.a. Franky De La Joya,Male,1994-10-23,None,None,Mexico,...,ILLICIT-DRUGS-EO14059; SDGT,Sanction,https://ofac.treasury.gov/recent-actions/20250811,United States Department of the Treasury,False,True,False,False,False,United States
1,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,Franky De La Joya,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"ESKEDA, Franki",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"a.k.a. ESQUEDA, Franky",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"a.k.a. ESKEDA, Franki",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"ESQUEDA, Franky",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","ALIZADEH HOSHYAR, Javad Saeid",Male,1976-08-24,None,None,Iran,...,NPWMD; IFSR,Sanction,https://ofac.treasury.gov/recent-actions/20250811,United States Department of the Treasury,False,True,False,False,False,United States
8,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADEH HUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADEH HOSHYAR, Javad Saeid",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [75]:
# final step adding sorted address including pob into temp_df_expanded4 as we have sorted both temp_df_expanded4 and df_sorted so it will aligned
concatenated_df = pd.concat([temp_df_expanded4, df_sorted], axis=1)
concatenated_df = concatenated_df.rename(columns={'name': 'entity_name','dates_of_birth':'date_of_birth_year'})
concatenated_df

,ID-NAME,ID,type,entity_name,alias_name,gender,date_of_birth_year,citizenship_country,citizenship_country_code,nationality_country,...,is_enforcement,is_apc,subject_country,address_type,address_street,address_city,address_province,address_postal_code,address_country,address_country_code
0,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,"ESQUEDA NIETO, Francisco Daniel",a.k.a. Franky De La Joya,Male,1994-10-23,None,None,Mexico,...,False,False,United States,NaN,None,Nuevo Laredo,Tamaulipas,NaN,Mexico,MEX
1,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,Franky De La Joya,None,None,None,None,None,...,None,None,None,Birth,"Nuevo Laredo, Tamaulipas",NaN,NaN,NaN,Mexico,MEX
2,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"ESKEDA, Franki",None,None,None,None,None,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"a.k.a. ESQUEDA, Franky",None,None,None,None,None,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"a.k.a. ESKEDA, Franki",None,None,None,None,None,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,None,"ESQUEDA, Franky",None,None,None,None,None,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"54505-ESQUEDA NIETO, Francisco Daniel",54505,Individual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","ALIZADEH HOSHYAR, Javad Saeid",Male,1976-08-24,None,None,Iran,...,False,False,United States,NaN,None,None,None,NaN,Iran,IRN
8,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADEH HUSHYAR, Javad",None,None,None,None,None,...,None,None,None,Birth,<NA>,NaN,NaN,NaN,Iran,IRN
9,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"a.k.a. ALIZADEH HOSHYAR, Javad Saeid",None,None,None,None,None,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
# importing format of ofac listing
df_format = pd.read_excel('OFAC_format.xlsx')
df_format

,ID,uid,merged_uids,type,entity_name,first_name,middle_name,last_name,alias_type,alias_name,...,Errors Update,Allocated to,Status (Error/Updated/No Change),Remarks,Date.1,UK SECO UN EU,Iden,Address,Title,full name


In [77]:
# Filter concatenated_df based on 'type'
df_org = concatenated_df[concatenated_df['type'] == 'Organization']
df_ind = concatenated_df[concatenated_df['type'] == 'Individual']

# Append to df_format to align columns and add missing columns as NaN
df_org_final = pd.concat([df_format, df_org], ignore_index=True, sort=False)
df_ind_final = pd.concat([df_format, df_ind], ignore_index=True, sort=False)

# Export to Excel with two sheets
with pd.ExcelWriter('final_output.xlsx') as writer:
    df_org_final.to_excel(writer, sheet_name='organization', index=False)
    df_ind_final.to_excel(writer, sheet_name='individual', index=False)